In [25]:
import numpy as np
import mne
from pyriemann.utils.base import logm, expm
from scipy.linalg import sqrtm, eigh
import sys

class GenerateurDonnees:
    def __init__(self, channels=3, sfreq=256.0, duration=10, change=5.0, A1=2.0e-6, A2=0.0e-6, sd=2.0e-6):
        self.channels = channels
        self.sfreq = sfreq
        self.duration = duration
        self.change = change
        self.frequencies = 10.0 + 2.0 * np.random.rand(channels)
        self.phases = 2.0 * np.pi * np.random.rand(channels)
        self.A1 = A1
        self.A2 = A2
        self.sd = sd
        self.info = mne.create_info([f'EEG{n:03}' for n in range(1, channels + 1)],
                                    ch_types=['eeg'] * channels, sfreq=sfreq)

    def generer_donnees(self):
        samples = int(self.sfreq * self.duration)
        t = np.linspace(0, self.duration, samples, endpoint=False)
        A = self.A1 + (t > self.change).astype(float) * (self.A2 - self.A1)
        data = A * np.cos(np.outer(self.frequencies, t) + np.outer(self.phases, np.ones(samples)))
        data += np.random.normal(0, self.sd, size=(self.channels, samples))
        # Vérification des valeurs infinies ou NaN
        if not np.isfinite(data).all():
            raise ValueError("Les données générées contiennent des valeurs infinies ou NaN")
        raw = mne.io.RawArray(data, self.info)
        return raw

    def exporter_donnees(self, filename='dummy.edf'):
        raw = self.generer_donnees()
        mne.export.export_raw(filename, raw, fmt='edf', overwrite=True)

# Classe CalculCovariance
class CalculCovariance:
    def __init__(self, p_points, stride):
        self.p_points = p_points
        self.stride = stride
    
    def calculer_matrices_covariance(self, donnees):
        k, n = donnees.shape
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            covariance_matrix = np.cov(segment) + np.eye(k) * 1e-6  # Régularisation ajoutée
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

# Classe Trajectoire
class Trajectoire:
    def __init__(self, *covariance_matrices):
        self.covariance_matrices = list(covariance_matrices)
        self.trajectoire = None
    
    def ajouter_covariance(self, nouvelle_covariance):
        self.covariance_matrices.append(nouvelle_covariance)
        self.trajectoire = None
    
    def calculer_trajectoire(self):
        self.trajectoire = sum(self.covariance_matrices)
    
    def obtenir_covariances(self):
        return self.covariance_matrices
    
    def obtenir_trajectoire(self):
        if self.trajectoire is None:
            self.calculer_trajectoire()
        return self.trajectoire
    
    def __str__(self):
        return (f'Trajectoire avec {len(self.covariance_matrices)} matrices de covariance\n'
                f'Trajectoire actuelle:\n{self.trajectoire}')

# Classe GestionnaireTrajectoires
class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelles_covariance):
        for nouvelle_covariance in nouvelles_covariance:
            if not np.isfinite(nouvelle_covariance).all():
                raise ValueError("La nouvelle covariance contient des valeurs infinies ou NaN")
            if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
                self.trajectoires.append(Trajectoire(nouvelle_covariance))
            else:
                while not len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire :
                    self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def tangent_space_projection(self, A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A
    
    def inverse_tangent_space_projection(self, ProjB_A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
        exp_transformed_ProjB_A = expm(transformed_ProjB_A)
        original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
        return original_A
    
    def transport_vector(self, Delta, A, B):
        inv_B = np.linalg.inv(B)
        A_invB = np.dot(A, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, Delta), E.T)
        return transported_Delta

    def projeter_et_transporter(self):
        if len(self.trajectoires) == 0:
            return

        for traj in self.trajectoires:
            covariances = traj.obtenir_covariances()
            projected_covariances = []

            for i in range(len(covariances) - 1):
                if not np.isfinite(covariances[i]).all() or not np.isfinite(covariances[i + 1]).all():
                    continue  # Skip matrices with inf or NaN values
                
                proj = self.tangent_space_projection(covariances[i], covariances[i + 1])
                if not np.isfinite(proj).all():
                    raise ValueError("La projection contient des valeurs infinies ou NaN")

                if projected_covariances:
                    for j in range(len(projected_covariances)):
                        projected_covariances[j] = self.transport_vector(projected_covariances[j], covariances[i], covariances[i + 1])
                        if not np.isfinite(projected_covariances[j]).all():
                            raise ValueError("Le vecteur transporté contient des valeurs infinies ou NaN")

                projected_covariances.append(proj)

            # Ajouter la dernière covariance sans projection car elle est déjà au bon endroit
            """if np.isfinite(covariances[-1]).all():
                projected_covariances.append(covariances[-1])"""

            # Remplacer les covariances dans la trajectoire par les covariances projetées et transportées
            traj.covariance_matrices = projected_covariances

            # Calculer la trajectoire après projection et transport
            traj.calculer_trajectoire()
            # Vérifier et régulariser les matrices pour qu'elles restent définies positives
            """for i in range(len(traj.covariance_matrices)):
                if not np.all(np.linalg.eigvals(traj.covariance_matrices[i]) > 0):
                    traj.covariance_matrices[i] += np.eye(traj.covariance_matrices[i].shape[0]) * 1e-6"""
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i+1}:\n{traj}\n'
        return result



# Main function
if __name__ == "__main__":
    # Rediriger la sortie vers un fichier
    sys.stdout = open('hamdoulah.txt', 'w')

    # Générer des données initiales
    generateur = GenerateurDonnees()
    raw_data_initiale = generateur.generer_donnees()
    donnees_initiales = raw_data_initiale.get_data()

    # Calculer les matrices de covariance initiales
    calcul_cov = CalculCovariance(p_points=256, stride=128)
    matrices_covariance_initiales = calcul_cov.calculer_matrices_covariance(donnees_initiales)

    # Créer le gestionnaire de trajectoires avec les matrices initiales
    gestionnaire = GestionnaireTrajectoires(5, *matrices_covariance_initiales)

    # Calculer la trajectoire initiale
    gestionnaire.projeter_et_transporter()
    print("Trajectoire initiale:")
    print(gestionnaire)

    # Générer de nouvelles données et ajouter les nouvelles matrices de covariance
    for _ in range(5):  # Ajout de 5 nouveaux segments de données
        raw_data_nouvelle = generateur.generer_donnees()
        donnees_nouvelles = raw_data_nouvelle.get_data()
        matrices_covariance_nouvelles = calcul_cov.calculer_matrices_covariance(donnees_nouvelles)
        
        gestionnaire.ajouter_covariance(matrices_covariance_nouvelles)
        """gestionnaire.projeter_et_transporter()
        print(f"Après ajout des nouvelles données {_ + 1}:")
        print(str(gestionnaire) + "\n")"""""""""

    # Rétablir la sortie standard vers la console
    sys.stdout = sys.__stdout__



In [4]:
from moabb.datasets import BNCI2014_001


In [7]:
if __name__ == "__main__":
    # Rediriger la sortie vers un fichier
    sys.stdout = open('hamdoulah1.txt', 'w')

    # Générer des données initiales
    # Exemple d'utilisation de la méthode
    dataset = BNCI2014_001()
    sessions = dataset.get_data(subjects=[1])
    subject = 1
    session_name = "0train"
    run_name = "0"
    raw = sessions[subject][session_name][run_name]
    donnees_initiales = raw.copy().crop(tmin=0,tmax=10).get_data()

    # Calculer les matrices de covariance initiales
    calcul_cov = CalculCovariance(p_points=256, stride=128)
    matrices_covariance_initiales = calcul_cov.calculer_matrices_covariance(donnees_initiales)

    # Créer le gestionnaire de trajectoires avec les matrices initiales
    gestionnaire = GestionnaireTrajectoires(5, *matrices_covariance_initiales)

    # Calculer la trajectoire initiale
    gestionnaire.projeter_et_transporter()
    print("Trajectoire initiale:")
    print(gestionnaire)

    # Générer de nouvelles données et ajouter les nouvelles matrices de covariance
    
    
    donnees_nouvelles = raw.copy().crop(tmin=10,tmax=11).get_data()
    matrices_covariance_nouvelles = calcul_cov.calculer_matrices_covariance(donnees_nouvelles)
        
    gestionnaire.ajouter_covariance(matrices_covariance_nouvelles)
    gestionnaire.projeter_et_transporter()
    print(f"Après ajout des nouvelles données {_ + 1}:")
    print(str(gestionnaire) + "\n")

    # Rétablir la sortie standard vers la console
    sys.stdout = sys.__stdout__

In [5]:
import numpy as np
from scipy.linalg import sqrtm, logm, expm
import logging

class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelles_covariance):
        for nouvelle_covariance in nouvelles_covariance:
            if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
                self.trajectoires.append(Trajectoire(nouvelle_covariance))
            else:
                while not len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
                    self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def tangent_space_projection(self, A, B):
        logging.debug(f"Tangent space projection: A = {A}, B = {B}")
        if not self.is_positive_definite(A) or not self.is_positive_definite(B):
            raise ValueError("Matrices must be positive definite")
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A
    
    def inverse_tangent_space_projection(self, ProjB_A, B):
        logging.debug(f"Inverse tangent space projection: ProjB_A = {ProjB_A}, B = {B}")
        if not self.is_positive_definite(B):
            raise ValueError("Matrix B must be positive definite")
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
        exp_transformed_ProjB_A = expm(transformed_ProjB_A)
        original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
        return original_A
    
    def transport_vector(self, Delta, A, B):
        logging.debug(f"Transport vector: Delta = {Delta}, A = {A}, B = {B}")
        if not self.is_positive_definite(A) or not self.is_positive_definite(B):
            raise ValueError("Matrices must be positive definite")
        inv_B = np.linalg.inv(B)
        A_invB = np.dot(A, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, Delta), E.T)
        return transported_Delta

    def is_positive_definite(self, matrix):
        try:
            np.linalg.cholesky(matrix)
            return True
        except np.linalg.LinAlgError:
            return False

    def projeter_et_transporter(self):
        if len(self.trajectoires) == 0:
            return

        for traj in self.trajectoires:
            covariances = traj.obtenir_covariances()
            projected_covariances = []

            for i in range(len(covariances) - 1):
                if not np.isfinite(covariances[i]).all() or not np.isfinite(covariances[i + 1]).all():
                    continue  # Skip matrices with inf or NaN values
                try:
                    proj = self.tangent_space_projection(covariances[i], covariances[i + 1])
                except ValueError as e:
                    print(f"Error in tangent_space_projection: {e}")
                    continue
                if projected_covariances:
                    for j in range(len(projected_covariances)):
                        try:
                            projected_covariances[j] = self.transport_vector(projected_covariances[j], covariances[i], covariances[i + 1])
                        except ValueError as e:
                            print(f"Error in transport_vector: {e}")
                            continue
                projected_covariances.append(proj)

            # Ajouter la dernière covariance sans projection car elle est déjà au bon endroit
            if np.isfinite(covariances[-1]).all():
                projected_covariances.append(covariances[-1])

            # Remplacer les covariances dans la trajectoire par les covariances projetées et transportées
            traj.covariance_matrices = projected_covariances

            # Calculer la trajectoire après projection et transport
            traj.calculer_trajectoire()
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i + 1}:\n{traj}\n'
        return result


In [6]:
if __name__ == "__main__":
    # Rediriger la sortie vers un fichier
    sys.stdout = open('hamdoulah.txt', 'w')

    # Générer des données initiales
    generateur = GenerateurDonnees()
    raw_data_initiale = generateur.generer_donnees()
    donnees_initiales = raw_data_initiale.get_data()

    # Calculer les matrices de covariance initiales
    calcul_cov = CalculCovariance(p_points=256, stride=128)
    matrices_covariance_initiales = calcul_cov.calculer_matrices_covariance(donnees_initiales)

    # Créer le gestionnaire de trajectoires avec les matrices initiales
    gestionnaire = GestionnaireTrajectoires(5, *matrices_covariance_initiales)

    # Calculer la trajectoire initiale
    gestionnaire.projeter_et_transporter()
    print("Trajectoire initiale:")
    print(gestionnaire)

    # Générer de nouvelles données et ajouter les nouvelles matrices de covariance
    for _ in range(5):  # Ajout de 5 nouveaux segments de données
        raw_data_nouvelle = generateur.generer_donnees()
        donnees_nouvelles = raw_data_nouvelle.get_data()
        matrices_covariance_nouvelles = calcul_cov.calculer_matrices_covariance(donnees_nouvelles)
        
        gestionnaire.ajouter_covariance(matrices_covariance_nouvelles)
        gestionnaire.projeter_et_transporter()
        print(f"Après ajout des nouvelles données {_ + 1}:")
        print(str(gestionnaire) + "\n")

    # Rétablir la sortie standard vers la console
    sys.stdout = sys.__stdout__